In [11]:
import os
import json

def fetch_data():
    with open(path, 'r') as f:
        train_data = [json.loads(line) for line in f]

    tag_mapping = {1: "NN", 2: "DT", 3: "JJ", 4: "OT"}

    # Preprocess training data
    train_sentences = []
    train_labels = []
    for entry in train_data:
        tokens = entry['tokens']
        pos_tags = entry['pos_tags']
        chunk_tags = entry['chunk_tags']
        
        train_sentences.append(pos_tags)
        train_labels.append(chunk_tags)
    
    return train_sentences, train_labels

train_data = fetch_data("./train.jsonl")
test_data = import_data("./test.jsonl")

In [10]:
train_sentences[1]

[1, 1]

In [23]:
import json
import numpy as np

class SingleRecurrentPerceptron:
    def __init__(self, input_size, output_size, learning_rate=0.1):
        self.input_size = input_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Initialize weights and bias
        self.weights = np.zeros((output_size, input_size))
        self.bias = np.zeros(output_size)
    
    def forward(self, inputs):
        return np.dot(self.weights, inputs)[:, np.newaxis] + self.bias[:, np.newaxis]
    
    def backward(self, inputs, target):
        prediction = self.forward(inputs)
        prediction = np.array(prediction)  # Ensure prediction is a NumPy array
        target = np.array(target)  # Ensure target is a NumPy array
        target = target[:, np.newaxis]  # Reshape target to match prediction's shape
        inputs = inputs[:, np.newaxis]  # Reshape inputs to match prediction's shape
        print("Target:", target)
        print("Prediction:", prediction)
        print("Inputs:", inputs)
        print("Target shape:", target.shape)
        print("Prediction shape:", prediction.shape)
        print("Inputs shape:", inputs.shape)
        error = target - prediction
        self.weights += self.learning_rate * error * inputs
        self.bias += self.learning_rate * error

        
    def train(self, inputs, targets, epochs):
        for _ in range(epochs):
            for i in range(len(inputs)):
                self.backward(inputs[i], targets[i])
    
    def predict(self, inputs):
        predictions = []
        for input_data in inputs:
            prediction = self.forward(input_data)
            predictions.append(prediction)
        return predictions

# Load training data
with open('train.jsonl', 'r') as f:
    train_data = [json.loads(line) for line in f]

# Map numerical values to POS tags
# Map numerical values to POS tags
tag_mapping = {1: "NN", 2: "DT", 3: "JJ", 4: "OT"}

# Calculate the number of unique POS tags
num_unique_tags = len(tag_mapping)

# Preprocess training data
# Preprocess training data
train_sentences = []
train_labels = []
for entry in train_data:
    tokens = entry['tokens']
    pos_tags = entry['pos_tags']
    chunk_tags = entry['chunk_tags']
    
    # Convert POS tags to one-hot encoded representation
    pos_tags_one_hot = np.zeros((len(pos_tags), num_unique_tags))
    for i, tag in enumerate(pos_tags):
        pos_tags_one_hot[i, tag - 1] = 1  # Subtract 1 to account for 0-based indexing
    
    # Flatten one-hot encoded representation
    flattened_tags = pos_tags_one_hot.flatten()
    
    train_sentences.append(flattened_tags)
    train_labels.append(chunk_tags)


# Initialize and train the single recurrent perceptron
# Initialize and train the single recurrent perceptron
input_size = len(train_sentences[0])  # Get input size from the first sample
output_size = 2  # Binary classification (1 for chunk, 0 for not chunk)
perceptron = SingleRecurrentPerceptron(input_size, output_size)
perceptron.train(train_sentences, train_labels, epochs=10)



# Load test data
with open('test.jsonl', 'r') as f:
    test_data = [json.loads(line) for line in f]

# Preprocess test data
test_sentences = []
test_labels = []
for entry in test_data:
    tokens = entry['tokens']
    pos_tags = entry['pos_tags']
    chunk_tags = entry['chunk_tags']
    
    # Convert POS tags to one-hot encoded representation
    pos_tags_one_hot = np.zeros((len(pos_tags), len(tag_mapping)))
    for i, tag in enumerate(pos_tags):
        pos_tags_one_hot[i, tag - 1] = 1  # Subtract 1 to account for 0-based indexing
    
    test_sentences.append(pos_tags_one_hot)
    test_labels.append(chunk_tags)

# Evaluate the trained perceptron
predictions = perceptron.predict(test_sentences)

# Assuming we have some evaluation function to compute accuracy
# Let's assume a simple accuracy calculation for demonstration
def accuracy(predictions, targets):
    correct = 0
    total = len(predictions)
    for pred, target in zip(predictions, targets):
        pred_labels = [1 if p > 0 else 0 for p in pred]
        if pred_labels == target:
            correct += 1
    return correct / total

acc = accuracy(predictions, test_labels)
print("Accuracy:", acc)


Target: [[1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]]
Prediction: [[0.]
 [0.]]
Inputs: [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]
Target shape: (9, 1)
Prediction shape: (2, 1)
Inputs shape: (36, 1)


ValueError: operands could not be broadcast together with shapes (9,1) (2,1) 